In [39]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import os

In [40]:
# Chargement du dataset
file_path = "./dataset_final.csv"
dataset = pd.read_csv(file_path, sep=";", encoding="utf-8")

# Affichage des premières lignes pour comprendre la structure
dataset.head()

,Unnamed: 0,geometry,label,date,tile_number,id,location_id,image_dir,chemin_directory,date_satellites_S1,...,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,timezone,folder
0,0,"[[[36.801563, -17.829673], [36.802546, -17.875...",0,06/02/2019,-91,sen12floods_s1_labels_0165_2019_02_06,165,../input/sen12flood/sen12flood/sen12floods_s1_...,'0165,20190206,...,18:10:59,1549469459,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQQL', '67283099999']",obs,Africa/Maputo,165
1,1,"[[[30.079766, -1.947669], [30.079851, -1.99392...",0,19/12/2018,26,sen12floods_s1_labels_0026_2018_12_19,26,../input/sen12flood/sen12flood/sen12floods_s1_...,'0026,20181219,...,18:03:40,1545235420,0.39,"Rain, Partially cloudy",Partly cloudy throughout the day with rain in ...,rain,"['HRYR', '64387099999', '64383099999']",obs,Africa/Kigali,26
2,2,"[[[-1.734277, 6.731243], [-1.734397, 6.684938]...",0,17/12/2018,100,sen12floods_s1_labels_0100_2018_12_17,100,../input/sen12flood/sen12flood/sen12floods_s1_...,'0100,20181217,...,17:54:57,1545069297,0.33,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,['65442099999'],obs,Africa/Accra,100
3,3,"[[[146.754028, -19.184316], [146.753959, -19.2...",1,11/02/2019,49,sen12floods_s1_labels_0305_2019_02_11,305,../input/sen12flood/sen12flood/sen12floods_s1_...,'0305,20190211,...,18:50:49,1549875049,0.21,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['95296099999', 'CBGR', 'YBTL', '94294099999',...",obs,Australia/Brisbane,305
4,4,"[[[48.748582, 30.637319], [48.749651, 30.59115...",0,06/04/2019,-10,sen12floods_s1_labels_0246_2019_04_06,246,../input/sen12flood/sen12flood/sen12floods_s1_...,'0246,20190406,...,19:36:46,1554563206,0.03,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['OIAM', 'OIAW', '40831099999', '40811099999',...",obs,Asia/Tehran,246


In [41]:
# Conversion de la colonne date en format datetime
if pd.api.types.is_object_dtype(dataset['date']):
    dataset['date'] = pd.to_datetime(dataset['date'], dayfirst=True)

# Trier le dataset par chemin_directory et date
dataset = dataset.sort_values(['chemin_directory', 'date'])

# Vérification des valeurs manquantes dans les colonnes importantes
for col in ['date', 'label', 'chemin_directory']:
    if dataset[col].isna().sum() > 0:
        print(f"Attention: {dataset[col].isna().sum()} valeurs manquantes dans la colonne '{col}'")

# Ajouter une colonne pour les jours écoulés depuis la première date de chaque série
dataset['days_elapsed'] = dataset.groupby('chemin_directory')['date'].transform(
    lambda x: (x - x.min()).dt.total_seconds() / (24*3600)
)

# Affichage du dataset après préparation
dataset.head()

,Unnamed: 0,geometry,label,date,tile_number,id,location_id,image_dir,chemin_directory,date_satellites_S1,...,sunsetEpoch,moonphase,conditions,description,icon,stations,source,timezone,folder,days_elapsed
107,331,"[[[34.811154, -19.680618], [34.811675, -19.726...",0,2019-02-06,0,sen12floods_s1_labels_0_2019_02_06,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190206,...,1549470078,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQBR', '67297099999']",obs,Africa/Maputo,0,0.0
239,1364,"[[[34.811154, -19.680618], [34.811675, -19.726...",0,2019-02-18,0,sen12floods_s1_labels_0_2019_02_18,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190218,...,1550506503,0.46,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQBR', '67297099999']",obs,Africa/Maputo,0,12.0
282,1755,"[[[34.811154, -19.680618], [34.811675, -19.726...",0,2019-03-02,0,sen12floods_s1_labels_0_2019_03_02,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190302,...,1551542803,0.86,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQBR', '67297099999']",obs,Africa/Maputo,0,24.0
331,282,"[[[34.811154, -19.680618], [34.811675, -19.726...",1,2019-03-13,0,sen12floods_s1_labels_0_2019_03_13,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190313,...,1552492672,0.23,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQBR', '67297099999']",obs,Africa/Maputo,0,35.0
1079,2999,"[[[34.811154, -19.680618], [34.811675, -19.726...",1,2019-03-14,0,sen12floods_s1_labels_0_2019_03_14,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190314,...,1552579021,0.25,"Rain, Partially cloudy",Partly cloudy throughout the day with rain in ...,rain,"['FQBR', '67297099999']",obs,Africa/Maputo,0,36.0


In [42]:
# Sélection des caractéristiques spécifiques pour l'entraînement
selected_features = [
    'tempmax', 'tempmin', 'temp', 'dew', 'humidity', 
    'precip', 'precipprob', 'preciptype', 'windgust', 'windspeed', 
    'winddir', 'pressure', 'cloudcover', 'visibility'
]

# Vérifier quelles caractéristiques sont effectivement disponibles dans le dataset
available_features = [col for col in selected_features if col in dataset.columns]
missing_features = [col for col in selected_features if col not in dataset.columns]

if missing_features:
    print(f"Attention: Les caractéristiques suivantes ne sont pas disponibles dans le dataset: {missing_features}")

print(f"Caractéristiques disponibles sélectionnées ({len(available_features)}): {available_features}")

# Nous utiliserons également 'days_elapsed' pour gérer les séries temporelles
if 'days_elapsed' not in available_features and 'days_elapsed' in dataset.columns:
    available_features.append('days_elapsed')
    print("La caractéristique 'days_elapsed' a été ajoutée pour gérer les intervalles de temps.")

# Ces caractéristiques seront utilisées pour l'entraînement du modèle LSTM
feature_columns = available_features

Caractéristiques disponibles sélectionnées (14): ['tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'precip', 'precipprob', 'preciptype', 'windgust', 'windspeed', 'winddir', 'pressure', 'cloudcover', 'visibility']
La caractéristique 'days_elapsed' a été ajoutée pour gérer les intervalles de temps.


In [43]:
# Séparation en séries temporelles individuelles
series_dict = {}
for directory in dataset['chemin_directory'].unique():
    # Extraction et tri de la série temporelle
    series = dataset[dataset['chemin_directory'] == directory].copy()
    series = series.sort_values('date')
    series_dict[directory] = series

print(f"Nombre de séries temporelles: {len(series_dict)}")

# Affichage d'un exemple de série
example_series_name = list(series_dict.keys())[0]
print(f"\nExemple de série temporelle '{example_series_name}':")
display(series_dict[example_series_name].head())

Nombre de séries temporelles: 336

Exemple de série temporelle ''0':


,Unnamed: 0,geometry,label,date,tile_number,id,location_id,image_dir,chemin_directory,date_satellites_S1,...,sunsetEpoch,moonphase,conditions,description,icon,stations,source,timezone,folder,days_elapsed
107,331,"[[[34.811154, -19.680618], [34.811675, -19.726...",0,2019-02-06,0,sen12floods_s1_labels_0_2019_02_06,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190206,...,1549470078,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQBR', '67297099999']",obs,Africa/Maputo,0,0.0
239,1364,"[[[34.811154, -19.680618], [34.811675, -19.726...",0,2019-02-18,0,sen12floods_s1_labels_0_2019_02_18,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190218,...,1550506503,0.46,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQBR', '67297099999']",obs,Africa/Maputo,0,12.0
282,1755,"[[[34.811154, -19.680618], [34.811675, -19.726...",0,2019-03-02,0,sen12floods_s1_labels_0_2019_03_02,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190302,...,1551542803,0.86,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQBR', '67297099999']",obs,Africa/Maputo,0,24.0
331,282,"[[[34.811154, -19.680618], [34.811675, -19.726...",1,2019-03-13,0,sen12floods_s1_labels_0_2019_03_13,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190313,...,1552492672,0.23,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['FQBR', '67297099999']",obs,Africa/Maputo,0,35.0
1079,2999,"[[[34.811154, -19.680618], [34.811675, -19.726...",1,2019-03-14,0,sen12floods_s1_labels_0_2019_03_14,0,../input/sen12flood/sen12flood/sen12floods_s1_...,'0,20190314,...,1552579021,0.25,"Rain, Partially cloudy",Partly cloudy throughout the day with rain in ...,rain,"['FQBR', '67297099999']",obs,Africa/Maputo,0,36.0


In [44]:
# Extraction des noms des séries
series_names = list(series_dict.keys())

# Stratification sur la présence d'inondations dans chaque série
has_floods = [series_dict[name]['label'].sum() > 0 for name in series_names]
print(f"Séries avec inondations: {sum(has_floods)}/{len(has_floods)}")

# Division des séries en ensembles d'entraînement et de test
try:
    train_series, test_series = train_test_split(
        series_names, test_size=0.2, random_state=42, 
        stratify=has_floods if len(set(has_floods)) > 1 else None
    )
except:
    train_series, test_series = train_test_split(
        series_names, test_size=0.2, random_state=42
    )

print(f"Division en {len(train_series)} séries d'entraînement et {len(test_series)} séries de test")

Séries avec inondations: 135/336
Division en 268 séries d'entraînement et 68 séries de test


In [45]:
def prepare_lstm_data_for_future_prediction(series, feature_columns, 
                                           sequence_length=5, prediction_horizon=1):
    """
    Prépare les données pour un modèle LSTM qui prédit les valeurs futures.
    
    Args:
        series (DataFrame): La série temporelle
        feature_columns (list): Liste des colonnes à utiliser comme caractéristiques
        sequence_length (int): Longueur des séquences pour le LSTM
        prediction_horizon (int): À quelle distance dans le futur prédire (1 = prochain pas)
        
    Returns:
        tuple: (X, y, scaler) où X sont les séquences d'entrée, y les valeurs cibles futures,
               et scaler l'objet pour dénormaliser les caractéristiques
    """
    # Ajouter days_elapsed aux caractéristiques pour gérer les intervalles irréguliers
    feature_columns_with_time = feature_columns.copy()
    if 'days_elapsed' not in feature_columns_with_time:
        feature_columns_with_time.append('days_elapsed')
    
    # Sélection des caractéristiques
    data = series[feature_columns_with_time].values
    
    # Récupérer les labels
    labels = series['label'].values
    
    # Normalisation des données (seulement pour les caractéristiques)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    
    # Création des séquences X et des cibles y (labels futurs)
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length - prediction_horizon + 1):
        X.append(scaled_data[i:i+sequence_length])
        # La cible est le label à prediction_horizon pas dans le futur
        y.append(labels[i+sequence_length+prediction_horizon-1])
    
    return np.array(X), np.array(y), scaler

In [46]:
# Modification des paramètres pour la préparation des données
sequence_length = 3  # Réduire de 5 à 3 observations consécutives nécessaires
prediction_horizon = 1

# Préparation des données d'entraînement
X_train_all, y_train_all = [], []

print("Préparation des données d'entraînement...")
for directory in train_series:
    series = series_dict[directory]
    # Vérifier que la série est assez longue (condition allégée)
    if len(series) <= sequence_length + prediction_horizon:
        print(f"⚠️ Série '{directory}' trop courte ({len(series)} échantillons). Ignorée.")
        continue
    
    # Préparation des données LSTM
    try:
        X, y, _ = prepare_lstm_data_for_future_prediction(
            series, feature_columns, 
            sequence_length=sequence_length, 
            prediction_horizon=prediction_horizon
        )
        
        if len(X) > 0:
            X_train_all.append(X)
            y_train_all.append(y)
            print(f"Série '{directory}': {len(X)} séquences, {np.sum(y)} inondations ({np.mean(y)*100:.1f}%)")
    except Exception as e:
        print(f"⚠️ Erreur avec la série '{directory}': {str(e)}")

Préparation des données d'entraînement...
⚠️ Erreur avec la série ''0143': could not convert string to float: "['rain']"
⚠️ Erreur avec la série ''0328': could not convert string to float: "['rain']"
⚠️ Erreur avec la série ''0332': could not convert string to float: "['rain']"
Série ''0248': 13 séquences, 0 inondations (0.0%)
⚠️ Erreur avec la série ''0122': could not convert string to float: "['rain']"
⚠️ Erreur avec la série ''0304': could not convert string to float: "['rain']"
Série ''0214': 12 séquences, 9 inondations (75.0%)
⚠️ Série ''0042' trop courte (3 échantillons). Ignorée.
⚠️ Erreur avec la série ''0335': could not convert string to float: "['rain']"
Série ''0262': 5 séquences, 0 inondations (0.0%)
Série ''21': 11 séquences, 9 inondations (81.8%)
⚠️ Erreur avec la série ''67': could not convert string to float: "['rain']"
Série ''0274': 5 séquences, 0 inondations (0.0%)
⚠️ Série ''0001' trop courte (4 échantillons). Ignorée.
Série ''0134': 6 séquences, 0 inondations (0.0%

d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  re


⚠️ Erreur avec la série ''0150': could not convert string to float: "['rain']"
⚠️ Erreur avec la série ''0156': could not convert string to float: "['rain']"
Série ''0012': 2 séquences, 0 inondations (0.0%)
Série ''0231': 11 séquences, 0 inondations (0.0%)
Série ''0130': 6 séquences, 0 inondations (0.0%)
⚠️ Erreur avec la série ''0023': could not convert string to float: "['rain']"
⚠️ Erreur avec la série ''0117': could not convert string to float: "['rain']"
Série ''40': 5 séquences, 0 inondations (0.0%)
Série ''0192': 11 séquences, 9 inondations (81.8%)
⚠️ Erreur avec la série ''46': could not convert string to float: "['rain']"
Série ''60': 6 séquences, 0 inondations (0.0%)
⚠️ Erreur avec la série ''0026': could not convert string to float: "['rain']"
⚠️ Série ''0061' trop courte (2 échantillons). Ignorée.
⚠️ Série ''0030' trop courte (3 échantillons). Ignorée.
Série ''65': 11 séquences, 0 inondations (0.0%)
Série ''0098': 7 séquences, 0 inondations (0.0%)
⚠️ Erreur avec la série '

d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  re

In [47]:
def prepare_lstm_data_with_robust_scaling(series, feature_columns, sequence_length=3, prediction_horizon=1):
    """
    Version améliorée de la préparation des données avec scaling robuste
    """
    # Copier les caractéristiques et ajouter days_elapsed
    feature_columns_with_time = feature_columns.copy()
    if 'days_elapsed' not in feature_columns_with_time:
        feature_columns_with_time.append('days_elapsed')
    
    # Sélection des caractéristiques
    data = series[feature_columns_with_time].values
    
    # Récupérer les labels
    labels = series['label'].values
    
    # Normalisation robuste des données (moins sensible aux valeurs aberrantes)
    from sklearn.preprocessing import RobustScaler
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(data)
    
    # Création des séquences avec chevauchement accru (stride=1)
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length - prediction_horizon + 1):
        X.append(scaled_data[i:i+sequence_length])
        y.append(labels[i+sequence_length+prediction_horizon-1])
    
    return np.array(X), np.array(y), scaler

def augment_minority_class(X_train, y_train, oversampling_factor=1.5):
    """
    Augmente la classe minoritaire (inondations) par des techniques simples
    """
    from scipy.interpolate import interp1d
    
    # Identifier les indices des exemples positifs (inondations)
    positive_indices = np.where(y_train == 1)[0]
    
    if len(positive_indices) == 0:
        return X_train, y_train
    
    # Nombre d'exemples positifs à ajouter
    n_to_add = int(len(positive_indices) * (oversampling_factor - 1))
    
    if n_to_add <= 0:
        return X_train, y_train
    
    # Sélectionner aléatoirement des exemples positifs pour l'augmentation
    aug_indices = np.random.choice(positive_indices, size=n_to_add)
    
    X_aug = []
    y_aug = []
    
    # Pour chaque exemple à augmenter
    for idx in aug_indices:
        x = X_train[idx].copy()
        
        # Ajouter un léger bruit gaussien pour la variation
        noise = np.random.normal(0, 0.05, x.shape)
        x_noisy = x + noise
        
        X_aug.append(x_noisy)
        y_aug.append(1)  # Classe positive
    
    # Combiner avec les données originales
    X_combined = np.vstack([X_train, np.array(X_aug)])
    y_combined = np.concatenate([y_train, np.array(y_aug)])
    
    return X_combined, y_combined

In [48]:
# Préparation des données de test
X_test_all, y_test_all = [], []
test_series_data = {}  # Pour stocker les séries de test séparément

print("Préparation des données de test...")
for directory in test_series:
    series = series_dict[directory]
    # Vérifier que la série est assez longue
    if len(series) <= sequence_length + prediction_horizon:
        print(f"⚠️ Série '{directory}' trop courte ({len(series)} échantillons). Ignorée.")
        continue
    
    # Préparation des données LSTM
    try:
        X, y, _ = prepare_lstm_data_for_future_prediction(
            series, feature_columns, 
            sequence_length=sequence_length, 
            prediction_horizon=prediction_horizon
        )
        
        if len(X) > 0:
            X_test_all.append(X)
            y_test_all.append(y)
            test_series_data[directory] = (X, y)
            print(f"Série '{directory}': {len(X)} séquences, {np.sum(y)} inondations ({np.mean(y)*100:.1f}%)")
    except Exception as e:
        print(f"⚠️ Erreur avec la série '{directory}': {str(e)}")

Préparation des données de test...
Série ''0277': 5 séquences, 0 inondations (0.0%)
⚠️ Erreur avec la série ''0324': could not convert string to float: "['rain']"
⚠️ Erreur avec la série ''0308': could not convert string to float: "['rain']"
⚠️ Erreur avec la série ''0325': could not convert string to float: "['rain']"
Série ''0257': 5 séquences, 0 inondations (0.0%)
Série ''0111': 17 séquences, 0 inondations (0.0%)
⚠️ Erreur avec la série ''0313': could not convert string to float: "['rain']"
Série ''0234': 11 séquences, 0 inondations (0.0%)
Série ''0135': 6 séquences, 0 inondations (0.0%)
Série ''0138': 7 séquences, 7 inondations (100.0%)
Série ''0221': 12 séquences, 9 inondations (75.0%)
⚠️ Série ''0067' trop courte (2 échantillons). Ignorée.
Série ''0203': 10 séquences, 8 inondations (80.0%)
⚠️ Erreur avec la série ''0004': could not convert string to float: "['rain']"
Série ''0223': 11 séquences, 0 inondations (0.0%)
⚠️ Erreur avec la série ''0027': could not convert string to flo

d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
d:\dataset\SEN12FLOOD (1)\env311\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  re

In [49]:
# Concaténation des données d'entraînement
X_train = np.vstack(X_train_all)
y_train = np.concatenate(y_train_all)
print(f"Données d'entraînement: {X_train.shape[0]} séquences, {X_train.shape[1:]} dimensions")
print(f"Distribution des labels d'entraînement: {np.sum(y_train)} inondations sur {len(y_train)} ({np.mean(y_train)*100:.1f}%)")

# Concaténation des données de test
X_test = np.vstack(X_test_all)
y_test = np.concatenate(y_test_all)
print(f"Données de test: {X_test.shape[0]} séquences, {X_test.shape[1:]} dimensions")
print(f"Distribution des labels de test: {np.sum(y_test)} inondations sur {len(y_test)} ({np.mean(y_test)*100:.1f}%)")

Données d'entraînement: 1016 séquences, (3, 15) dimensions
Distribution des labels d'entraînement: 329 inondations sur 1016 (32.4%)
Données de test: 289 séquences, (3, 15) dimensions
Distribution des labels de test: 98 inondations sur 289 (33.9%)


In [50]:
# Calcul des poids des classes pour gérer le déséquilibre
if np.sum(y_train == 1) > 0:
    ratio = np.sum(y_train == 0) / np.sum(y_train == 1)
    class_weight = {0: 1, 1: ratio}
    print(f"Poids des classes: {class_weight}")
else:
    class_weight = None
    print("⚠️ Pas d'exemples positifs dans les données d'entraînement!")

Poids des classes: {0: 1, 1: np.float64(2.088145896656535)}


In [51]:
def create_lstm_model(input_shape, lstm_units=256, dropout_rate=0.3, learning_rate=0.0005, bidirectional=True):
    """
    Version optimisée du modèle LSTM avec attention et régularisation améliorée
    """
    from tensorflow.keras.layers import Input, Attention, Concatenate, LayerNormalization
    from tensorflow.keras.regularizers import l1_l2
    
    # Input layer
    inputs = Input(shape=input_shape)
    
    # Première couche LSTM avec régularisation L1L2
    if bidirectional:
        x = Bidirectional(LSTM(lstm_units, 
                              return_sequences=True, 
                              recurrent_dropout=0.2,
                              kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))(inputs)
    else:
        x = LSTM(lstm_units, 
                 return_sequences=True, 
                 recurrent_dropout=0.2,
                 kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(inputs)
    
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    # Seconde couche LSTM
    if bidirectional:
        x = Bidirectional(LSTM(lstm_units//2, 
                              return_sequences=True,
                              recurrent_dropout=0.2,
                              kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))(x)
    else:
        x = LSTM(lstm_units//2, 
                 return_sequences=True,
                 recurrent_dropout=0.2,
                 kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(x)
    
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    # Troisième couche LSTM
    if bidirectional:
        lstm_out = Bidirectional(LSTM(lstm_units//4))(x)
    else:
        lstm_out = LSTM(lstm_units//4)(x)
    
    # Couches denses
    x = LayerNormalization()(lstm_out)
    x = Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    # Création du modèle
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compilation avec métriques
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', Precision(), Recall(), AUC()]
    )
    
    return model


def create_optimized_lstm_model(input_shape):
    """
    Version plus complexe du modèle LSTM avec des couches supplémentaires
    """
    from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
    from tensorflow.keras.regularizers import l1_l2
    
    inputs = Input(shape=input_shape)
    
    # Première couche LSTM
    x = Bidirectional(LSTM(384, return_sequences=True, recurrent_dropout=0.3))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    # Deuxième couche LSTM
    x = Bidirectional(LSTM(256, return_sequences=True, recurrent_dropout=0.3))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    # Troisième couche LSTM 
    x = Bidirectional(LSTM(128))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    # Couches denses
    x = Dense(128, activation='relu', kernel_regularizer=l1_l2(1e-5, 1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    optimizer = Adam(learning_rate=0.0003)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', Precision(), Recall(), AUC()]
    )
    
    return model

In [ ]:
model = create_optimized_lstm_model(
    input_shape=(X_train.shape[1], X_train.shape[2]),
    lstm_units=512,  # Augmenté pour une meilleure capacité d'apprentissage
    dropout_rate=0.3,
    learning_rate=0.0008,
    bidirectional=True
)
# Entraînement du modèle avec des données augmentées

X_train_aug, y_train_aug = augment_minority_class(X_train, y_train, oversampling_factor=1.5)


# Configuration améliorée des callbacks
def get_optimized_callbacks():
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=15,  # Augmenté de 10 à 15 pour permettre plus d'apprentissage
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            filepath='best_lstm_model.keras',
            monitor='val_loss',
            save_best_only=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            verbose=1,
            min_lr=0.00001
        ),
        tf.keras.callbacks.TensorBoard(
            log_dir='./logs',
            histogram_freq=1
        )
    ]
    return callbacks

callbacks = get_optimized_callbacks()

# Configuration d'entraînement améliorée
def train_with_cosine_decay():
    # Définir une fonction d'apprentissage cyclique avec décroissance en cosinus
    initial_learning_rate = 0.001
    lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
        initial_learning_rate,
        first_decay_steps=10,
        t_mul=2.0,
        m_mul=0.9,
        alpha=0.0001
    )
    
    optimizer = Adam(learning_rate=lr_schedule)
    
    # Adapter le modèle avec le nouvel optimiseur
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', Precision(), Recall(), AUC()]
    )
    
    # Entraînement du modèle avec batch dynamique et paramètres optimaux
    history = model.fit(
        X_train, y_train,
        epochs=150,  # Augmenté pour permettre plus d'apprentissage
        batch_size=16,  # Réduit pour plus de stabilité
        validation_data=(X_val, y_val),
        callbacks=get_optimized_callbacks(),
        verbose=1,
        class_weight=class_weight
    )
    
    return history

TypeError: create_optimized_lstm_model() got an unexpected keyword argument 'lstm_units'

In [ ]:
# Ensemble de modèles
def create_model_ensemble(X_train, y_train, X_val, y_val, input_shape):
    models = []
    
    # Modèle 1: LSTM standard
    model1 = create_lstm_model(input_shape)
    models.append(model1)
    
    # Modèle 2: LSTM avec plus de couches
    model2 = create_optimized_lstm_model(input_shape) 
    models.append(model2)
    
    # Modèle 3: LSTM avec paramètres différents
    model3 = create_lstm_model(input_shape, lstm_units=384, dropout_rate=0.2)
    models.append(model3)
    
    # Entraîner chaque modèle
    for i, model in enumerate(models):
        print(f"Entraînement du modèle {i+1}/{len(models)}")
        model.fit(
            X_train, y_train,
            epochs=100,
            batch_size=32,
            validation_data=(X_val, y_val),
            callbacks=get_optimized_callbacks(),
            verbose=1,
            class_weight=class_weight
        )
    
    # Fonction pour combiner les prédictions
    def ensemble_predict(X):
        predictions = []
        for model in models:
            pred = model.predict(X)
            predictions.append(pred)
        
        # Moyenne des prédictions
        return np.mean(np.array(predictions), axis=0)
    
    return ensemble_predict

In [ ]:
# Validation croisée temporelle
from sklearn.model_selection import TimeSeriesSplit

def time_series_cross_validation(X, y, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
        print(f"Training fold {fold+1}/{n_splits}")
        
        # Split the data
        X_train_fold, X_test_fold = X[train_idx], X[test_idx]
        y_train_fold, y_test_fold = y[train_idx], y[test_idx]
        
        # Train the model
        model = create_lstm_model(input_shape=(X.shape[1], X.shape[2]))
        
        model.fit(
            X_train_fold, y_train_fold,
            epochs=50,
            batch_size=32,
            validation_data=(X_test_fold, y_test_fold),
            verbose=0
        )
        
        # Evaluate
        results = model.evaluate(X_test_fold, y_test_fold, verbose=0)
        
        # Store metrics
        fold_metrics.append({metric: value for metric, value in zip(model.metrics_names, results)})
    
    # Calculate average metrics across folds
    avg_metrics = {}
    for metric in fold_metrics[0].keys():
        avg_metrics[metric] = np.mean([fold[metric] for fold in fold_metrics])
    
    return avg_metrics

In [53]:
# Ajouter des métriques d'évaluation plus complètes
def evaluate_model_extensively(model, X_test, y_test):
    """
    Évaluation complète du modèle avec diverses métriques et visualisations
    """
    from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

    # Prédictions sur le jeu de test
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Évaluation standard
    results = model.evaluate(X_test, y_test, verbose=0)
    metrics_names = model.metrics_names
    
    print("Résultats de l'évaluation:")
    for metric_name, value in zip(metrics_names, results):
        print(f"  - {metric_name}: {value:.4f}")
    
    # Rapport de classification détaillé
    print("\nRapport de classification:")
    print(classification_report(y_test, y_pred, target_names=['Pas d\'inondation', 'Inondation']))
    
    # Matrice de confusion
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=['Pas d\'inondation', 'Inondation'],
               yticklabels=['Pas d\'inondation', 'Inondation'])
    plt.xlabel('Prédiction')
    plt.ylabel('Valeur réelle')
    plt.title('Matrice de confusion')
    plt.show()
    
    # Courbe Précision-Rappel
    plt.figure(figsize=(8, 6))
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
    plt.plot(recall, precision, marker='.')
    plt.xlabel('Rappel')
    plt.ylabel('Précision')
    plt.title('Courbe Précision-Rappel')
    plt.grid(True)
    plt.show()
    
    # Courbe ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taux de faux positifs')
    plt.ylabel('Taux de vrais positifs')
    plt.title('Courbe ROC')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

In [54]:
# Calcul des poids des classes pour gérer le déséquilibre
if np.sum(y_train == 1) > 0:
    # Ratio ajusté pour éviter de trop favoriser la classe minoritaire
    ratio = min(np.sum(y_train == 0) / np.sum(y_train == 1), 10.0)  # Limitation du ratio à 10
    class_weight = {0: 1.0, 1: ratio}
    print(f"Poids des classes: {class_weight}")
else:
    class_weight = None
    print("⚠️ Pas d'exemples positifs dans les données d'entraînement!")

Poids des classes: {0: 1.0, 1: np.float64(2.088145896656535)}


In [ ]:
# Évaluation du modèle sur les données de test
results = model.evaluate(X_test, y_test, verbose=0)
metrics_names = model.metrics_names

print("Résultats de l'évaluation:")
for metric_name, value in zip(metrics_names, results):
    print(f"  - {metric_name}: {value:.4f}")

Résultats de l'évaluation:
  - loss: 0.6929
  - compile_metrics: 0.6109
